In [1]:
#Importing all the required libraries
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import re
import operator
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer

In [2]:
#Reading the data from excel Dataframes and concatinating all the sheets
df1 = pd.read_excel("C:\Machine Learning\Sentimental Analysis\comments.csv", sheetname='test1')
df2 = pd.read_excel("C:\Machine Learning\Sentimental Analysis\comments.csv", sheetname='test2')
df3 = pd.read_excel("C:\Machine Learning\Sentimental Analysis\comments.csv", sheetname='test3')
df4 = pd.read_excel("C:\Machine Learning\Sentimental Analysis\comments.csv", sheetname='test4')
frames = [df1,df2,df3,df4]
#Original data
df = pd.concat(frames)
df_new = df[['SOURCE','TONE','COMMENT']]


In [3]:
# Remove NAN and 
df_new.isnull().sum()
nan_rows = df_new[df_new['TONE'].isnull()]
nan_rows.tail(1)
df_nan_removed = df_new.dropna()
df_nan_removed.count()
df_nan_removed.isnull().sum()
#Remove Neutral Comments for first trail
df_remove_neutral = df_nan_removed[df_nan_removed['TONE']== 'Neutral']
df_clean = df_nan_removed.drop(df_remove_neutral.index,axis=0)
df_clean.count()
df_clean.groupby('TONE').count()
# Replacing Positive and Negative with 1 and 0
value_map = {'Positive': 1, 'Negative': 0}
df_integers = df_clean.replace({'TONE': value_map})
#Selecting REquired Coloumns TONE and Comment
df_integers_a = df_integers[['TONE','COMMENT']]
df_integers_a.head(1)
# Replacing the values insisde brackets in the comemnts section, this needs to be applied on each element separetly
#thus we will use apply method and a lambda function 
dfa = df_integers_a['COMMENT'].apply(lambda x: re.sub(r"\(.*\) ", "", x))
#Merging with Original DF
df_naive_bayes = pd.concat([df_integers_a['TONE'],dfa], axis =1)
#Regex to remove all other characters except Alphabets
dfb = df_naive_bayes['COMMENT'].apply(lambda x: re.sub ('[^A-Za-z\s]+', '', x))
df_final = pd.concat ([df_integers_a['TONE'],dfb], axis =1)
stop = stopwords.words('english')
df_stopwords_removed = df_final['COMMENT'].apply(lambda  x: ' '.join([word for word in x.split() if word not in (stop)]))
df_final = pd.concat ([df_integers_a['TONE'],df_stopwords_removed], axis =1)

In [13]:
#Applying Naive Bayes
X_train1 = df_final['COMMENT']
Y_train1 = df_final['TONE']

In [16]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train1, Y_train1, random_state=1)
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(X_train)
vector_test = vectorizer.transform(X_test)
nb = MultinomialNB()
nb.fit(vector,y_train)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(157,)
(53,)
(157,)
(53,)


In [17]:
y_pred_class = nb.predict(vector_test)

In [18]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.75471698113207553

In [20]:
print(y_test.value_counts())

0    39
1    14
Name: TONE, dtype: int64


AttributeError: 'NoneType' object has no attribute 'head'

In [22]:
#Null Accuracy
y_test.value_counts().head(1)/len(y_test)

0    0.735849
Name: TONE, dtype: float64

In [23]:
metrics.confusion_matrix(y_test,y_pred_class)

array([[38,  1],
       [12,  2]], dtype=int64)

In [26]:

X_test[y_pred_class > y_test]
X_test[(y_pred_class==1) & (y_test==0)]

394    crap post Junk offered
Name: COMMENT, dtype: object

In [27]:
X_test[y_pred_class < y_test]
X_test[(y_pred_class==0) & (y_test==1)]

14     Ive literally heard no one talking about this ...
146    Hi there LOVE what you guys did with the iPhon...
133    My son inherited his grandfathers truck Thanks...
148         Works greati can track my  year old all over
148    Awesome app and services My mom has a jeep che...
401    Who knew My family ad I have this app and I ha...
92                                    Its a good product
235    This is a TERRIFIC addition for your car I lov...
193          No worries thank you Henry Have a great day
446    I believe thay hum is the best thing dor peopl...
203    Re Superbowl The girl in the middle is great a...
186                       I have on my Acura TLits great
Name: COMMENT, dtype: object

In [28]:
X_test[394]

'crap post Junk offered'

In [35]:
y_pred_prob = nb.predict_proba(vector_test)[:, 1]
y_pred_prob

metrics.roc_auc_score(y_test, y_pred_prob)


0.79120879120879117